In [2]:
import polars as pl
import json
import os

In [3]:
# Load furniture categories
source_folder = "3D-FUTURE-scene"
threedfuture_file_path = source_folder + '/train/train_set.json'

with open(threedfuture_file_path, 'r') as json_file:
    data = json.load(json_file)

for key in data:
    print(key, len(data[key]))

categories 34
images 14761
annotations 73513


In [4]:
# Create Dataframe tables
# Categories
categories_data = data['categories']
df_categories = pl.DataFrame(categories_data)

df_categories.head()

id,super-category,category
i64,str,str
1,"""Cabinet/Shelf/…","""Children Cabin…"
2,"""Cabinet/Shelf/…","""Nightstand"""
3,"""Cabinet/Shelf/…","""Bookcase / jew…"
4,"""Cabinet/Shelf/…","""Wardrobe"""
5,"""Cabinet/Shelf/…","""Coffee Table"""


In [5]:
# Images
images_data = data['images']
df_images = pl.DataFrame(images_data)

df_images.head()

id,width,height,file_name
i64,i64,i64,str
1,1200,1200,"""0000000"""
2,1200,1200,"""0000001"""
3,1200,1200,"""0000002"""
4,1200,1200,"""0000003"""
5,1200,1200,"""0000004"""


In [6]:
source_folder = "./"
file_path = source_folder + 'blip_anotation_3D_DS.csv'

In [7]:
df = pl.read_csv(file_path)

df.head()

,annotation_id,image_id,category_id,area,bbox,model_id,texture_id,fov,style,theme,material,pose,occlusion,width,height,file_name,caption,Room
i64,i64,i64,i64,i64,str,str,str,f64,str,str,str,str,str,i64,i64,i64,str,str
0,1,1,33,13073,"""[526.0, 56.0, …","""da64b6ef-d704-…","""da64b6ef-d704-…",0.958973,"""Modern""",null,null,"""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom"""
1,2,1,2,8014,"""[1140.0, 795.0…","""4d5ff861-d5ce-…","""4d5ff861-d5ce-…",null,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom"""
2,3,1,23,24436,"""[0.0, 603.0, 2…","""a939be24-9c3d-…","""a939be24-9c3d-…",0.958973,"""Light Luxury""","""Gold Foil""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom"""
3,4,1,4,211496,"""[173.0, 287.0,…","""1d9f21cd-359f-…","""1d9f21cd-359f-…",0.958973,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom"""
4,5,1,13,235956,"""[414.0, 553.0,…","""433123cb-172e-…","""433123cb-172e-…",0.958973,"""Mediterranean""","""Striped Grid""","""Rough Cloth""","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom"""


In [8]:
df.group_by("image_id", )

In [9]:
df_joined = df.join(df_categories, left_on="category_id", right_on="id")
df_joined.head()

,annotation_id,image_id,category_id,area,bbox,model_id,texture_id,fov,style,theme,material,pose,occlusion,width,height,file_name,caption,Room,super-category,category
i64,i64,i64,i64,i64,str,str,str,f64,str,str,str,str,str,i64,i64,i64,str,str,str,str
0,1,1,33,13073,"""[526.0, 56.0, …","""da64b6ef-d704-…","""da64b6ef-d704-…",0.958973,"""Modern""",null,null,"""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Lighting""","""Pendant Lamp"""
1,2,1,2,8014,"""[1140.0, 795.0…","""4d5ff861-d5ce-…","""4d5ff861-d5ce-…",null,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom""","""Cabinet/Shelf/…","""Nightstand"""
2,3,1,23,24436,"""[0.0, 603.0, 2…","""a939be24-9c3d-…","""a939be24-9c3d-…",0.958973,"""Light Luxury""","""Gold Foil""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom""","""Table""","""Dressing Table…"
3,4,1,4,211496,"""[173.0, 287.0,…","""1d9f21cd-359f-…","""1d9f21cd-359f-…",0.958973,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Cabinet/Shelf/…","""Wardrobe"""
4,5,1,13,235956,"""[414.0, 553.0,…","""433123cb-172e-…","""433123cb-172e-…",0.958973,"""Mediterranean""","""Striped Grid""","""Rough Cloth""","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Bed""","""King-size Bed"""


In [10]:
df_concat = df_joined.with_columns(
    pl.concat_str(
        [
            pl.col("style").fill_null(""),
            pl.col("material").fill_null("").str.to_lowercase(),
            pl.col("category").str.to_lowercase(),
        ],
        separator=" ",
    ).alias("model_description"),
)

df_concat.head()

,annotation_id,image_id,category_id,area,bbox,model_id,texture_id,fov,style,theme,material,pose,occlusion,width,height,file_name,caption,Room,super-category,category,model_description
i64,i64,i64,i64,i64,str,str,str,f64,str,str,str,str,str,i64,i64,i64,str,str,str,str,str
0,1,1,33,13073,"""[526.0, 56.0, …","""da64b6ef-d704-…","""da64b6ef-d704-…",0.958973,"""Modern""",null,null,"""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Lighting""","""Pendant Lamp""","""Modern pendan…"
1,2,1,2,8014,"""[1140.0, 795.0…","""4d5ff861-d5ce-…","""4d5ff861-d5ce-…",null,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom""","""Cabinet/Shelf/…","""Nightstand""","""Korean composi…"
2,3,1,23,24436,"""[0.0, 603.0, 2…","""a939be24-9c3d-…","""a939be24-9c3d-…",0.958973,"""Light Luxury""","""Gold Foil""","""Composite Boar…","""{'translation'…","""N/A""",1200,1200,0,"""a bedroom with…","""bedroom""","""Table""","""Dressing Table…","""Light Luxury c…"
3,4,1,4,211496,"""[173.0, 287.0,…","""1d9f21cd-359f-…","""1d9f21cd-359f-…",0.958973,"""Korean""","""Smooth Net""","""Composite Boar…","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Cabinet/Shelf/…","""Wardrobe""","""Korean composi…"
4,5,1,13,235956,"""[414.0, 553.0,…","""433123cb-172e-…","""433123cb-172e-…",0.958973,"""Mediterranean""","""Striped Grid""","""Rough Cloth""","""{'translation'…","""NO""",1200,1200,0,"""a bedroom with…","""bedroom""","""Bed""","""King-size Bed""","""Mediterranean …"


# Concatenate room captions
# For each image
# BLIP descriptions + The room has {style} {category}, {style} {category}, ...
# e.g. The room has Mediterranean style Coffee table, Korean style Sofa

In [13]:
df_img = df_concat.group_by(['image_id', "Room", "caption"]).agg(
    pl.col("model_description").unique().str.concat(", "),
    pl.col("model_description").unique().count().alias("item_count")
).join(
    df_images.select(pl.exclude(["width", "height"])), left_on="image_id", right_on="id"
)

# Concatenate the generated annotations with BLIP captions
df_img = df_img.with_columns(
    pl.concat_str([
        pl.col("caption").str.strip_chars(),
        pl.col("model_description").str.strip_chars(", ")
    ], separator="; ").alias("full_description")
)

print(df_img)

shape: (14_761, 7)
┌──────────┬─────────────┬────────────────┬───────────────┬────────────┬───────────┬───────────────┐
│ image_id ┆ Room        ┆ caption        ┆ model_descrip ┆ item_count ┆ file_name ┆ full_descript │
│ ---      ┆ ---         ┆ ---            ┆ tion          ┆ ---        ┆ ---       ┆ ion           │
│ i64      ┆ str         ┆ str            ┆ ---           ┆ u32        ┆ str       ┆ ---           │
│          ┆             ┆                ┆ str           ┆            ┆           ┆ str           │
╞══════════╪═════════════╪════════════════╪═══════════════╪════════════╪═══════════╪═══════════════╡
│ 1        ┆ bedroom     ┆ a bedroom with ┆ Korean        ┆ 5          ┆ 0000000   ┆ a bedroom     │
│          ┆             ┆ a bed,         ┆ composite     ┆            ┆           ┆ with a bed,   │
│          ┆             ┆ dressers, …    ┆ board         ┆            ┆           ┆ dressers, …   │
│          ┆             ┆                ┆ wardrobe,…    ┆            ┆

In [15]:
df_img.write_csv(file="3dfront-annotated.csv")